# MyMethods.jl -- Examples

This notebook illustrates basic functionalities of the MyMethods.jl package.

### Installation and Uninstallation of MyMethods.jl

In [1]:
# Install the package from GitHub (first time only) 
#using Pkg; 
#Pkg.add(url="https://github.com/thomaswiemann/MyMethods.jl");

# Uninstall 
#Pkg.rm("MyMethods")

In [1]:
# Specify module directory (depending on GitHub location...)
push!(LOAD_PATH, "$(homedir())\\Documents\\GitHub")
push!(LOAD_PATH, "$(homedir())\\GitHub")

# Load MyMethods.jl into your Julia session 
using MyMethods

┌ Info: Precompiling MyMethods [18de7e97-ce2a-4ea9-8d3b-e4b0c3b0b515]
└ @ Base loading.jl:1278
┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
│ - If you have MyMethods checked out for development and have
│   added DataFrames as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MyMethods
└ Loading DataFrames into MyMethods from project dependency, future warnings for MyMethods are suppressed.


In [6]:
# Load additionally useful (but not necessary) packages
using Statistics # for a quantile function
using Plots; plotly() # for a plotting backend

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\thomas\.julia\packages\Plots\D7Ica\src\backends.jl:373


Plots.PlotlyBackend()

### myLS
myLS() generates a (weighted) least squares object.

In [13]:
# Example data
N = 1000 # sample size
beta = [1,2,-1,4] # regression coefficients
X = hcat(ones(N), rand(N,3)); # regressor matrix
y = X*beta + randn(N,1); # outcome simulation

In [14]:
# Estimate least squares fit
fit_LS = myLS(y, X);

# Calculate and print standard errors
inference(fit_LS);

,coef,se,t-stat,p-val
,Float64,Float64,Float64,Float64
1,1.12433,0.101535,11.0733,1.68988e-28
2,2.02428,0.109809,18.4345,6.94447e-76
3,-1.04317,0.110164,-9.46928,2.81785e-21
4,3.80134,0.109375,34.7551,1.16168e-264


### myLLR
myLLR generates a local linear regression object.

In [131]:
# Example data
N = 1000 # sample size
beta = [1,2,-1,4] # regression coefficients
_r = (1 .+ 1.5 .* (rand(N,1) .- 0.5)); # running variable
X = hcat(ones(N), rand(N,3)); # regressor matrix
y = X*beta.*_r + randn(N,1); # outcome simulation

In [132]:
# Estimate local constant regression with bandwidth h = 0.4
_x = quantile(_r[:], collect(1:100)./100) # values at which to fit
fit_LLR = myLLR(y, _r, # response and running variable
    X[:, 2:end],# additional variables to include
    _x = _x, 
    K=0, h=0.4); # degree and bandwidth

In [135]:
_x

100-element Array{Float64,1}:
 0.2656484092579677
 0.2774906033682134
 0.29307653700181474
 0.30439374863742424
 0.3200715324138433
 0.33728895193506764
 0.3626667523226489
 0.3849546115166412
 0.4067482723722035
 0.41995037225799425
 0.4331349314890906
 0.4489111317453007
 0.4602229813355726
 ⋮
 1.5667975906946192
 1.5821195080826043
 1.596622720292868
 1.6136589314376988
 1.6338662100674632
 1.6445089541461901
 1.6589552648396477
 1.680063852245586
 1.6926106390892857
 1.7054541338685374
 1.7258876788597826
 1.74775320658851

In [133]:
# Calculate the local coefficients for quantiles of the running variable
coef_LLR = coef(fit_LLR);

In [134]:
# Plot coefficients as a function of the running variable
plot(fit_LLR._x, coef_LLR,
title = "Local linear regression coefficients",
label = nothing) # plot estimates
plot!(fit_LLR._x, beta'.*_x,
label = nothing) # plot true coefficients

<!DOCTYPE html>
 
 
 Plots.jl

# mybootstrap

In [128]:
# bootstrap myLS
res = mybootstrap(myLS, y, 100, X, data_args = [1], 
    get = coef, red =  hcat);

In [129]:
reduce_boot(res)

4×100 Array{Float64,2}:
  0.692152   0.626168   0.689002  …   0.644184   0.620782   0.45542
  1.74164    2.20252    2.33886       2.01935    1.88418    2.30356
 -0.6882    -1.12338   -1.28467      -0.637578  -0.737501  -0.454245
  4.4628     4.59024    4.6475        4.48297    4.35518    4.22594

In [130]:
# bootstrap myLLR
res = mybootstrap(myLLR, y, 10, _r, X[:, 2:end], data_args = [1, 2],
    _x = quantile(_r[:], collect(1:10)./10), 
    get = coef)
reduce_boot(res)

10×4×10 Array{Float64,3}:
[:, :, 1] =
 0.290555  1.08074  -0.387792  2.21233
 0.318712  1.26848  -0.426589  2.51783
 0.419302  1.38701  -0.43125   2.94681
 0.679461  1.60093  -0.5846    3.44774
 0.911018  1.99837  -0.885739  4.06526
 1.16156   2.32003  -1.31119   4.79951
 1.38514   2.74163  -1.68857   5.33133
 1.58761   2.86956  -1.91858   5.6361
 1.66551   2.87468  -1.95394   6.05327
 1.93414   2.78145  -2.09427   6.20668

[:, :, 2] =
 0.513896  0.795941  -0.702422  2.34904
 0.624628  1.00603   -0.864156  2.65625
 0.692087  1.31262   -0.980484  3.1155
 0.853743  1.70592   -1.1459    3.56753
 1.18854   2.00423   -1.43022   3.97878
 1.33584   2.27036   -1.55538   4.5518
 1.44165   2.64254   -1.83104   5.41849
 1.50833   2.81924   -2.01859   6.09503
 1.5257    3.0133    -2.09773   6.61833
 1.54943   3.14489   -2.1464    6.9692

[:, :, 3] =
 -0.124429   1.66478  -0.259502  2.47704
 -0.125528   1.83884  -0.292834  2.825
  0.0838497  1.86532  -0.473635  3.18392
  0.567599   1.90612  -0.9259

### Parallel Computing

In [14]:
using Distributed

In [15]:
addprocs(3)

3-element Array{Int64,1}:
 2
 3
 4

In [16]:
@everywhere using MyMethods

      From worker 4:	┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
      From worker 4:	│ - If you have MyMethods checked out for development and have
      From worker 4:	│   added DataFrames as a dependency but haven't updated your primary
      From worker 4:	│   environment's manifest file, try `Pkg.resolve()`.
      From worker 4:	│ - Otherwise you may need to report an issue with MyMethods
      From worker 4:	└ Loading DataFrames into MyMethods from project dependency, future warnings for MyMethods are suppressed.
      From worker 2:	┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
      From worker 2:	│ - If you have MyMethods checked out for development and have
      From worker 2:	│   added DataFrames as a dependency but haven't updated your primary
      From worker 2:	│   environment's manifest file, try `Pkg.resolve()`.
      From worker 2:	│ - Otherwise you may need to report an issue with MyMethods
      From worke

In [17]:
# bootstrap myLS
res = mybootstrapPAR(myLS, y, 10, X, data_args = [1], 
    get = coef, red = hcat∘vcat)

10×1 Array{Array{Float64,2},2}:
 [0.8457001054630248; 2.1825671470351113; -0.8944855934348915; 4.04688690718834]
 [0.9373090862909891; 2.0940918769346597; -0.948752636120286; 4.017162520394206]
 [0.8507148666019242; 2.0749954268338637; -0.8830533994991726; 4.12602531607754]
 [0.8014808274913243; 2.080395695759484; -0.7761127133526732; 4.0846830416763495]
 [0.9176496288512465; 2.0578995711870784; -0.8539456377419344; 3.9783865214218768]
 [1.001301130535213; 1.9205582435279795; -0.8569072105544677; 3.9739084282577966]
 [0.9167132934554411; 2.126301679851184; -0.945166007690668; 3.9219204827835905]
 [0.8548611528638019; 2.067775315143256; -0.8849871095247795; 4.074145801982866]
 [0.9704688194402704; 1.949345550706462; -0.9909546567402508; 4.119448906026626]
 [0.9081846920883201; 2.185545488657733; -0.9866043708865838; 4.013858069747636]

In [33]:
## Parallel coefficient function for myLLR object
function coefPAR2(fit::myLLR, _x=fit._x;
        dynamic=false)
    # Data parameters
    N_x = length(_x)
    # Check whether additional variables are included 
    with_control = !isnothing(fit.control)
    if with_control
        dim_coef = size(fit.control,2) + fit.K + 1
    else
        dim_coef = fit.K + 1
    end
    
    # Run LLR in parallel
    if !dynamic # w/o dynamic job scheduling
        coef_mat = @distributed (hcat) for idx in 1:length(_x)
            coef(fit, _x[idx])'
        end
    else # w/ dynamic job scheduling
        coef_mat = Array{Float64, 2}(undef, N_x, dim_coef)
        @sync begin
            for p in workers()
                @async begin
                    for idx in 1:length(_x)
                        coef_mat[idx,:] = remotecall_fetch(coef, p, 
                        fit, _x[idx])'
                    end
                end
            end
        end
    end
    
    # Return local coefficients
    return coef_mat'
end #COEFPAR.MYLLR

coefPAR2 (generic function with 2 methods)

In [34]:
# bootstrap myLLR
coef_LLR = coefPAR2(fit_LLR)

LoadError: TaskFailedException:
On worker 2:
TypeError: in new, expected Int64, got a value of type Array{Float64,2}
deserialize at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Serialization\src\Serialization.jl:1356
handle_deserialize at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Serialization\src\Serialization.jl:837
deserialize at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Serialization\src\Serialization.jl:1350
handle_deserialize at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Serialization\src\Serialization.jl:837
deserialize at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Serialization\src\Serialization.jl:773 [inlined]
deserialize_msg at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Distributed\src\messages.jl:99
#invokelatest#1 at .\essentials.jl:710 [inlined]
invokelatest at .\essentials.jl:709 [inlined]
message_handler_loop at D:\buildbot\work

## KNN matching

In [1]:
# Specify module directory (depending on GitHub location...)
push!(LOAD_PATH, "$(homedir())\\Documents\\GitHub")
push!(LOAD_PATH, "$(homedir())\\GitHub")

# Load MyMethods.jl into your Julia session 
using MyMethods

┌ Info: Precompiling MyMethods [18de7e97-ce2a-4ea9-8d3b-e4b0c3b0b515]
└ @ Base loading.jl:1278
┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
│ - If you have MyMethods checked out for development and have
│   added DataFrames as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MyMethods
└ Loading DataFrames into MyMethods from project dependency, future warnings for MyMethods are suppressed.


In [2]:
# Data parameters
N = 1000 # sample size
beta = [1,3] # regression coefficients

# Data
x = rand(N,1) # variable to match on
D = rand(0:1, N); # running variable
X = hcat(ones(N), D); # regressor matrix
y = X*beta + randn(N,1); # outcome simulation

K = 5
replacement = true

true

In [3]:
res = myMatch(y, D, x)

myMatch(3.1044679433165614, 2.9832305173822196, 3.0417881941085065, [1.0068915680976585; 1.343611000632665; … ; 0.6745053781333361; 1.2341350929983772], [0, 0, 1, 1, 0, 1, 1, 1, 1, 0  …  1, 1, 1, 0, 0, 0, 0, 1, 0, 0], [0.42346425588753545; 0.49143631369064233; … ; 0.597034603701702; 0.7129720300923583], 1, true)

### mySieve

In [4]:
N = 1000
z = rand(N,1)
x = 2 .* (1 .- 2 .* z) # \sim U[-2, 2]
y = sin.(2 .* x) .+ 2 .* exp.(-16 .* x.^2) .+ 3 .* randn(N,1);

K = 3;

In [5]:
res = mySieve(y, x, basis="LSplines")

mySieve([1.0153037045894362; 1.0018229198603452; … ; 3.6896691205177974; -2.6716514245205913], [0.42596232207459295; 0.8949939370773986; … ; -1.744524360912596; 0.9560023058297156], [1.0 0.0552374339371271 … -0.0 -0.0; 1.0 -1.0883370605844735 … -0.0 -0.0; … ; 1.0 -0.8446142241239549 … -0.0 -0.0; 1.0 -0.026299278666534143 … -0.0 -0.0], "LSplines", 3, #undef)

### mySplines

In [171]:
using Statistics

In [183]:
r = quantile(x[:], collect(1:(K))./(K+1))

3-element Array{Float64,1}:
 -0.9332992761689949
 -0.022016838769304492
  0.9016281952267564

In [184]:
collect(1:(K))./(K+1)

3-element Array{Float64,1}:
 0.25
 0.5
 0.75

In [185]:
[(x.>r[k]).*(x.-r[k]) for k in 1:K]

3-element Array{Array{Float64,2},1}:
 [2.025396229234434; 2.5167799773415602; … ; 2.285982527966956; 1.9490962500829008]
 [1.1141137918347432; 1.6054975399418696; … ; 1.3747000905672655; 1.0378138126832104]
 [0.19046875783868233; 0.6818525059458087; … ; 0.4510550565712046; 0.11416877868714947]